In [1]:
import os
import elasticsearch as es
from dotenv import load_dotenv

In [2]:
load_dotenv()

True

In [3]:
username = os.getenv("USERNAME")
password = os.getenv("PASSWORD")
client = es.Elasticsearch("http://10.11.12.69:9200", http_auth=(username, password))

In [4]:
client.ping()

True

In [5]:
client.indices.get_alias()

/usr/local/Caskroom/miniconda/base/envs/ADP/lib/python3.9/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: this request accesses system indices: [.async-search, .kibana_task_manager_7.15.1_001, .fleet-policies-7, .security-7, .tasks, .transform-internal-007, .apm-custom-link, .kibana_7.15.1_001, .kibana_security_session_1, .apm-agent-configuration], but in a future major version, direct access to system indices will be prevented by default
  warnings.warn(message, category=ElasticsearchWarning)
/usr/local/Caskroom/miniconda/base/envs/ADP/lib/python3.9/site-packages/elasticsearch/connection/base.py:209: ElasticsearchWarning: this request accesses aliases with names reserved for system indices: [.kibana_task_manager_7.15.1, .kibana_7.15.1, .kibana_task_manager, .security, .data-frame-internal-3, .kibana, .fleet-policies], but in a future major version, direct access to system indices and their aliases will not be allowed
  warnings.warn(message, category=Elastics

{'kibana_sample_data_flights': {'aliases': {}},
 '.async-search': {'aliases': {}},
 '.monitoring-logstash-7-2021.12.01': {'aliases': {}},
 'constantin_amazondataproject': {'aliases': {}},
 'logstash-2021.11.08-000001': {'aliases': {'logstash': {'is_write_index': True}}},
 '%{[@metadata][dest_index]}': {'aliases': {}},
 'kibana_sample_data_logs': {'aliases': {}},
 '.kibana_task_manager_7.15.1_001': {'aliases': {'.kibana_task_manager': {},
   '.kibana_task_manager_7.15.1': {}}},
 '.items-default-000001': {'aliases': {'.items-default': {'is_write_index': True}}},
 '.transform-notifications-000002': {'aliases': {'.transform-notifications-read': {'is_hidden': True}}},
 'kismet_alerts': {'aliases': {}},
 '.fleet-policies-7': {'aliases': {'.fleet-policies': {}}},
 '.monitoring-kibana-7-2021.11.26': {'aliases': {}},
 'haussner_kismet_devices': {'aliases': {}},
 '.monitoring-es-7-2021.11.25': {'aliases': {}},
 '.monitoring-es-7-2021.11.26': {'aliases': {}},
 'kibana_sample_data_ecommerce': {'al

In [6]:
index_name = "constantin_amazondataproject_v3"

In [7]:
client.indices.create(index=index_name)

{'acknowledged': True,
 'shards_acknowledged': True,
 'index': 'constantin_amazondataproject_v3'}

In [8]:
client.search(index=index_name, query={"match_all": {}})

{'took': 0,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [9]:
import pandas as pd

In [10]:
import csv
import numpy as np

In [11]:
body = {
    "mappings": {
        "properties": {
            "marketplace": { "type": "keyword" },
            "customer_id": { "type": "keyword"},
            "review_id": { "type": "keyword" },
            "product_id": { "type": "keyword" },
            "product_parent": { "type": "keyword" },
            "product_title": { "type": "text" },
            "star_rating": { "type": "byte" },
            "helpful_votes": { "type": "integer" },
            "total_votes": { "type": "integer" },
            "vine": { "type": "boolean" },
            "verified_purchase": { "type": "boolean" },
            "review_headline": { "type": "text" },
            "review_body": { "type": "text" },
            "review_date": { "type": "date" }
        }
    }
}

In [12]:
convert_to_bool = lambda x: True if x == "Y" else False
converters = {"vine": convert_to_bool, "verified_purchase": convert_to_bool}

In [13]:
dtypes = {
    "marketplace": "category",
    "customer_id": np.int64,
    "review_id": "object",
    "product_id": "object",
    "product_parent": np.int64,
    "product_title": "object",
    "star_rating": np.int8,
    "helpful_votes": np.int16,
    "total_votes": np.int16,
    "review_headline": "object",
    "review_body": "object",
}

In [14]:
def data_iterator(reader):
        # need to remove when using csv.DictReader
        for chunk in reader:
            chunk.replace(np.nan, "", inplace=True)
            for _, row in chunk.iterrows():
                doc = {
                    "marketplace": row["marketplace"],
                    "customer_id": row["customer_id"],
                    "review_id": row["review_id"],
                    "product_id": row["product_id"],
                    "product_parent": row["product_parent"],
                    "product_title": row["product_title"],
                    "product_category": row["product_category"],
                    "star_rating": row["star_rating"],
                    "helpful_votes": row["helpful_votes"],
                    "total_votes": row["total_votes"],
                    "vine": row["vine"],
                    "verified_purchase": row["verified_purchase"],
                    "review_headline": row["review_headline"],
                    "review_body": row["review_body"],
                    "review_date": row["review_date"],
                }
                yield doc

In [15]:
client.delete_by_query(index=index_name, body={"query": {"match_all": {}}})

{'took': 0,
 'timed_out': False,
 'total': 0,
 'deleted': 0,
 'batches': 0,
 'version_conflicts': 0,
 'noops': 0,
 'retries': {'bulk': 0, 'search': 0},
 'throttled_millis': 0,
 'requests_per_second': -1.0,
 'throttled_until_millis': 0,
 'failures': []}

In [16]:
client.search({"query": {"match_all": {}}}, index=index_name)

/var/folders/ps/dgdwqdr11mq34k0cc13_s_940000gn/T/ipykernel_53782/3111317702.py:1: DeprecationWarning: Using positional arguments for APIs is deprecated and will be disabled in 8.0.0. Instead use only keyword arguments for all APIs. See https://github.com/elastic/elasticsearch-py/issues/1698 for more information
  client.search({"query": {"match_all": {}}}, index=index_name)


{'took': 0,
 'timed_out': False,
 '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0},
 'hits': {'total': {'value': 0, 'relation': 'eq'},
  'max_score': None,
  'hits': []}}

In [33]:
from elasticsearch.helpers import bulk
with open("../amazon_reviews_multilingual_DE_v1_00.tsv", mode="r") as f:
    # TODO: replace this with a version of csv.DictReader
    reader = pd.read_csv(
        f,
        delimiter="\t",
        quoting=csv.QUOTE_NONE,
        encoding="utf-8",
        chunksize=7000,
        parse_dates=["review_date"],
        converters=converters,
        dtype=dtypes,
    )
    res = bulk(client=client, index=index_name, actions=data_iterator(reader))
res

KeyboardInterrupt: 

In [34]:
with open("../amazon_reviews_multilingual_DE_v1_00.tsv", mode="r") as f:
    reader = pd.read_csv(
        f,
        delimiter="\t",
        quoting=csv.QUOTE_NONE,
        encoding="utf-8",
        chunksize=7000,
        parse_dates=["review_date"],
        converters=converters,
        dtype=dtypes,
    )
    for chunk in reader:
        l = chunk
        break

In [41]:
test = l.to_json(orient="records")

In [50]:
pd.DataFrame.from_dict([{"Test": np.nan, "Test2": 2}]).to_json(orient="records")

'[{"Test":null,"Test2":2}]'